<a href="https://colab.research.google.com/github/Jaeji/AM360Paper/blob/main/Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# ลบโฟลเดอร์เดิมทิ้งก่อน เพื่อป้องกันกรณีไฟล์มาไม่ครบ
!rm -rf AM360Paper

# Clone ข้อมูลมาใหม่
!git clone https://github.com/Jaeji/AM360Paper.git

# เช็คไฟล์ว่ามาครบไหม
import os
print("Checking .mat files:")
for root, dirs, files in os.walk("./AM360Paper"):
    for f in files:
        if f.endswith(".mat"):
            print(f)

Cloning into 'AM360Paper'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 53.98 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking .mat files:
B0007.mat
B0018.mat
B0005.mat
B0006.mat


# Section 1: Import ไลบรารีและเตรียมฟังก์ชันอ่านไฟล์

In [44]:
import numpy as np
import pandas as pd
import os, time
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import warnings

# ปิด Warning เพื่อความสะอาดของ Output
warnings.filterwarnings('ignore')

# ฟังก์ชันค้นหาไฟล์ในโฟลเดอร์
def find_mat_file(filename, search_path='./AM360Paper'):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# ฟังก์ชันดึง Feature จากไฟล์ .mat
def extract_features(mat_path, battery_name):
    mat = loadmat(mat_path)
    # หา key ที่ถูกต้องในไฟล์ mat
    if battery_name in mat:
        battery = mat[battery_name]
    else:
        keys = [k for k in mat.keys() if not k.startswith('__')]
        battery = mat[keys[0]]

    cycles = battery[0][0]["cycle"][0]
    rows = []
    cycle_id = 1

    for c in cycles:
        # เลือกเฉพาะข้อมูล discharge ที่สมบูรณ์
        if c["type"][0] == "discharge" and "data" in c.dtype.names:
            d = c["data"][0][0]
            if "Capacity" in d.dtype.names and d["Capacity"].size > 0:
                rows.append([
                    battery_name,
                    cycle_id,
                    np.mean(d["Voltage_measured"][0]),
                    np.std(d["Voltage_measured"][0]),
                    np.mean(d["Current_measured"][0]),
                    np.std(d["Current_measured"][0]),
                    np.mean(d["Temperature_measured"][0]),
                    d["Capacity"][0][0]
                ])
                cycle_id += 1

    return pd.DataFrame(
        rows,
        columns=["battery", "cycle", "V_mean", "V_std", "I_mean", "I_std", "Temp_mean", "Capacity"]
    )

# Section 2: โหลดข้อมูลจากไฟล์ .mat

In [45]:
batteries = ["B0005", "B0006", "B0007", "B0018"]
dfs = []

print("Loading data...")
for b in batteries:
    path = find_mat_file(f"{b}.mat")
    if path:
        print(f"- Found {b} at {path}")
        dfs.append(extract_features(path, b))
    else:
        print(f"- Warning: {b}.mat not found")

if not dfs:
    raise FileNotFoundError("ไม่พบไฟล์ข้อมูล .mat กรุณาตรวจสอบโฟลเดอร์ AM360Paper อีกครั้ง")

df = pd.concat(dfs, ignore_index=True)
print(f"Data loaded successfully. Total rows: {len(df)}")

Loading data...
- Found B0005 at ./AM360Paper/B0005.mat
- Found B0006 at ./AM360Paper/B0006.mat
- Found B0007 at ./AM360Paper/B0007.mat
- Found B0018 at ./AM360Paper/B0018.mat
Data loaded successfully. Total rows: 636


# Section 3: คำนวณ RUL และเตรียมข้อมูล Train/Test

In [46]:
# 1. คำนวณ RUL (Remaining Useful Life) หน่วยเป็นจำนวนรอบ (Cycles)
df["RUL"] = df.groupby("battery")["cycle"].transform(lambda x: x.max() - x)

# 2. แบ่ง Train/Test (ตาม Paper ใช้ B0018 เป็น Test set)
train_df = df[df["battery"] != "B0018"]
test_df  = df[df["battery"] == "B0018"]

# Fallback: กรณีไฟล์ B0018 หายไป ให้แบ่งท้าย B0005 มาเทสแทน
if len(test_df) == 0:
    print("Note: B0018 not found. Using part of B0005 for testing.")
    test_df = train_df[train_df["battery"] == "B0005"].tail(50)
    train_df = train_df.drop(test_df.index)

# 3. เตรียม X, y และทำการ Scaling
features = ["V_mean", "V_std", "I_mean", "I_std", "Temp_mean", "Capacity"]
X_train = train_df[features].values
y_train = train_df["RUL"].values
X_test  = test_df[features].values
y_test  = test_df["RUL"].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

Train set: (504, 6), Test set: (132, 6)


# Section 4: กำหนด Model และ Hyperparameters

In [47]:
models = {
    "Lasso": Lasso(alpha=0.1),
    "Random_Forest": RandomForestRegressor(n_estimators=100, max_depth=12, random_state=42),
    "Gradient_Boosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42),
    "Kernel_Ridge": KernelRidge(alpha=1.0, kernel="rbf", gamma=0.01),
    "Decision_Tree": DecisionTreeRegressor(max_depth=8, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, objective="reg:squarederror", random_state=42)
}

# Section 5: เทรนโมเดลและคำนวณค่า Error

In [48]:
# เตรียม List เก็บผลลัพธ์
results_table2 = [] # RMSE, SSE
results_table3 = [] # Total Error
results_table4 = [] # Relative Error
results_table5 = [] # Time

# ลำดับโมเดลตามตารางในรูปภาพ
model_order = ["Lasso", "Random_Forest", "Gradient_Boosting", "Kernel_Ridge", "Decision_Tree", "XGBoost"]

print("Starting Training & Evaluation...")

for name in model_order:
    model = models[name]

    # จับเวลาและเทรน
    start_time = time.time()
    model.fit(X_train, y_train)
    run_time = time.time() - start_time

    # ทำนายผล
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # --- คำนวณ Metrics ---

    # RMSE & SSE
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    sse_train = np.sum((y_train - y_train_pred) ** 2)
    sse_test = np.sum((y_test - y_test_pred) ** 2)

    # Total Error (Sum Absolute Error)
    total_err_train = np.sum(np.abs(y_train - y_train_pred))
    total_err_test = np.sum(np.abs(y_test - y_test_pred))

    # Relative Error (ระวังการหารด้วย 0)
    mask_train, mask_test = y_train > 0, y_test > 0
    rel_err_train = np.mean(np.abs(y_train[mask_train] - y_train_pred[mask_train]) / y_train[mask_train])
    rel_err_test = np.mean(np.abs(y_test[mask_test] - y_test_pred[mask_test]) / y_test[mask_test])

    # บันทึกผล
    results_table2.append([name, rmse_train, rmse_test, sse_train, sse_test])
    results_table3.append([name, total_err_train, total_err_test])
    results_table4.append([name, rel_err_train, rel_err_test])
    results_table5.append([name, run_time])

    print(f"Finished: {name}")

Starting Training & Evaluation...
Finished: Lasso
Finished: Random_Forest
Finished: Gradient_Boosting
Finished: Kernel_Ridge
Finished: Decision_Tree
Finished: XGBoost


# Section 6: แสดงผลลัพธ์ตาราง (Table II - V)

In [49]:
from IPython.display import display

# Table II: RMSE & SSE
print("\nTABLE II: RMSE AND SSE VALUES")
df_table2 = pd.DataFrame(results_table2, columns=["Methods", "RMSE Train", "RMSE Test", "SSE Train", "SSE Test"])
display(df_table2.round(2))

# Table III: Total Error
print("\nTABLE III: TOTAL ERROR VALUES")
df_table3 = pd.DataFrame(results_table3, columns=["Methods", "Total Error Train", "Total Error Test"])
display(df_table3.round(2))

# Table IV: Relative Error
print("\nTABLE IV: AVERAGE RELATIVE ERROR VALUES")
df_table4 = pd.DataFrame(results_table4, columns=["Methods", "Relative Error Train", "Relative Error Test"])
display(df_table4.round(4))

# Table V: Running Time
print("\nTABLE V: RUNNING TIMES")
df_table5 = pd.DataFrame(results_table5, columns=["Methods", "Time (second)"])
display(df_table5.round(4))


TABLE II: RMSE AND SSE VALUES


,Methods,RMSE Train,RMSE Test,SSE Train,SSE Test
0,Lasso,8.73,14.24,38403.97,26759.26
1,Random_Forest,1.94,16.84,1889.24,37445.18
2,Gradient_Boosting,1.71,17.41,1473.59,39991.61
3,Kernel_Ridge,13.66,20.30,94044.10,54395.15
4,Decision_Tree,2.10,24.31,2230.51,78020.14
5,XGBoost,0.20,16.01,21.02,33837.49



TABLE III: TOTAL ERROR VALUES


,Methods,Total Error Train,Total Error Test
0,Lasso,3389.92,1535.99
1,Random_Forest,617.65,1836.79
2,Gradient_Boosting,641.49,1998.07
3,Kernel_Ridge,4950.80,2356.30
4,Decision_Tree,632.47,2460.23
5,XGBoost,71.73,1807.07



TABLE IV: AVERAGE RELATIVE ERROR VALUES


,Methods,Relative Error Train,Relative Error Test
0,Lasso,0.3429,0.6876
1,Random_Forest,0.0521,0.4726
2,Gradient_Boosting,0.0452,0.4173
3,Kernel_Ridge,0.5455,1.1677
4,Decision_Tree,0.0521,0.5018
5,XGBoost,0.0044,0.3865



TABLE V: RUNNING TIMES


,Methods,Time (second)
0,Lasso,0.0020
1,Random_Forest,0.5631
2,Gradient_Boosting,0.6429
3,Kernel_Ridge,0.1257
4,Decision_Tree,0.0125
5,XGBoost,0.5954
